```mermaid
classDiagram
    class TransactionStatsCalculator {
        +time_bins: List[int]
        +metrics: List[str]
        +ratio_pairs: List[Tuple]
        +calculate_window_stats(df, reference_age, window_days)
        +calculate_ratios(stats)
        +calculate_transaction_stats(df, current_transaction)
        +process_transactions(df)
        -_validate_inputs(df)
        -_initialize_columns(df)
    }

    class ConfigParams {
        +amount_col: str
        +customer_id_col: str
        +age_col: str
    }

    class StatisticsResult {
        +volume_Nd_avg: float
        +volume_Nd_std: float
        +volume_Nd_count: int
        +volume_ratios: Dict
    }

    TransactionStatsCalculator -- ConfigParams
    TransactionStatsCalculator -- StatisticsResult
```

In [ ]:
import pandas as pd
import numpy as np
from typing import List, Dict, Union, Tuple
from datetime import datetime, timedelta

class TransactionStatsCalculator:
    def __init__(
        self,
        time_bins: List[int] = [7, 14, 30],
        metrics: List[str] = ['avg', 'std', 'count'],
        ratio_pairs: List[Tuple[int, int]] = [(7, 14), (7, 30), (14, 30)]
    ):
        """
        Initialize the transaction statistics calculator
        
        Parameters:
        -----------
        time_bins : List[int]
            List of time windows in days to calculate metrics for
        metrics : List[str]
            List of metrics to calculate
        ratio_pairs : List[tuple]
            List of time bin pairs to calculate ratios for
        """
        self.time_bins = sorted(time_bins)
        self.metrics = metrics
        self.ratio_pairs = ratio_pairs
        self._validate_ratio_pairs()
        
    def _validate_ratio_pairs(self) -> None:
        """Validate that ratio pairs use existing time bins"""
        all_periods = set(self.time_bins)
        for from_days, to_days in self.ratio_pairs:
            if from_days not in all_periods or to_days not in all_periods:
                raise ValueError(
                    f"Ratio pair ({from_days}, {to_days}) uses undefined time bin"
                )

    def _validate_inputs(
        self,
        df: pd.DataFrame,
        amount_col: str,
        customer_id_col: str,
        age_col: str
    ) -> None:
        """Validate input DataFrame and column names"""
        required_cols = [amount_col, customer_id_col, age_col]
        missing_cols = [col for col in required_cols if col not in df.columns]
        if missing_cols:
            raise ValueError(f"Missing required columns: {missing_cols}")

    def calculate_window_stats(
        self,
        df: pd.DataFrame,
        reference_age: float,
        window_days: int,
        amount_col: str = 'amount',
        age_col: str = 'account_age'
    ) -> Dict[str, float]:
        """
        Calculate statistics for a specific time window using account age
        """
        # Calculate window boundaries (up to day-1)
        end_age = reference_age - 1  # day-1
        start_age = end_age - window_days
        
        # Filter data for the time window
        mask = (df[age_col] <= end_age) & (df[age_col] > start_age)
        window_data = df.loc[mask, amount_col]
        
        stats = {}
        if len(window_data) > 0:
            if 'avg' in self.metrics:
                stats[f'volume_{window_days}d_avg'] = float(window_data.mean())
            if 'std' in self.metrics:
                stats[f'volume_{window_days}d_std'] = float(window_data.std()) if len(window_data) > 1 else 0.0
            if 'count' in self.metrics:
                stats[f'volume_{window_days}d_count'] = int(len(window_data))
        else:
            # Handle empty windows
            for metric in self.metrics:
                stats[f'volume_{window_days}d_{metric}'] = 0.0
                
        return stats

    def calculate_ratios(
        self,
        stats: Dict[str, float]
    ) -> Dict[str, float]:
        """Calculate ratios between different time periods"""
        ratio_stats = {}
        
        for from_days, to_days in self.ratio_pairs:
            # Volume average ratio
            if 'avg' in self.metrics:
                ratio_name = f'volume_avg_{from_days}d_to_{to_days}d_ratio'
                from_avg = stats.get(f'volume_{from_days}d_avg', 0)
                to_avg = stats.get(f'volume_{to_days}d_avg', 0)
                ratio_stats[ratio_name] = from_avg / to_avg if to_avg != 0 else 0
                
            # Count ratio (normalized by time period)
            if 'count' in self.metrics:
                ratio_name = f'volume_count_{from_days}d_to_{to_days}d_ratio'
                from_count = stats.get(f'volume_{from_days}d_count', 0)
                to_count = stats.get(f'volume_{to_days}d_count', 0)
                normalized_ratio = (from_count / from_days) / (to_count / to_days) if to_count != 0 else 0
                ratio_stats[ratio_name] = normalized_ratio
                
        return ratio_stats

    def calculate_transaction_stats(
        self,
        df: pd.DataFrame,
        current_transaction: pd.Series,
        amount_col: str = 'amount',
        customer_id_col: str = 'customer_id',
        age_col: str = 'account_age'
    ) -> Dict[str, float]:
        """Calculate all statistics for a single transaction"""
        # Get customer's transactions up to this point
        customer_df = df[
            (df[customer_id_col] == current_transaction[customer_id_col]) &
            (df[age_col] < current_transaction[age_col])
        ].copy()
        
        # Calculate stats for each time bin
        all_stats = {}
        for days in self.time_bins:
            window_stats = self.calculate_window_stats(
                df=customer_df,
                reference_age=current_transaction[age_col],
                window_days=days,
                amount_col=amount_col,
                age_col=age_col
            )
            all_stats.update(window_stats)
        
        # Calculate ratios
        ratio_stats = self.calculate_ratios(all_stats)
        all_stats.update(ratio_stats)
        
        return all_stats

    def _initialize_columns(self, df: pd.DataFrame) -> pd.DataFrame:
        """Initialize all metric columns in the DataFrame"""
        # Generate column names
        metric_columns = [
            f'volume_{days}d_{metric}'
            for days in self.time_bins
            for metric in self.metrics
        ]
        ratio_columns = [
            f'volume_{metric}_{from_d}d_to_{to_d}d_ratio'
            for from_d, to_d in self.ratio_pairs
            for metric in ['avg', 'count'] if metric in self.metrics
        ]
        
        # Initialize columns with zeros
        for col in metric_columns + ratio_columns:
            df[col] = 0.0
            
        return df

    def process_transactions(
        self,
        df: pd.DataFrame,
        amount_col: str = 'amount',
        customer_id_col: str = 'customer_id',
        age_col: str = 'account_age'
    ) -> pd.DataFrame:
        """Process all transactions and add statistics columns"""
        # Validate inputs
        self._validate_inputs(df, amount_col, customer_id_col, age_col)
        
        # Sort and copy data
        df = df.sort_values([customer_id_col, age_col]).copy()
        
        # Initialize metric columns
        df = self._initialize_columns(df)
        
        # Calculate metrics for each transaction
        for idx, row in df.iterrows():
            stats = self.calculate_transaction_stats(
                df=df,
                current_transaction=row,
                amount_col=amount_col,
                customer_id_col=customer_id_col,
                age_col=age_col
            )
            
            # Update row with calculated stats
            for metric, value in stats.items():
                df.at[idx, metric] = value
        
        return df

    def get_column_names(self) -> List[str]:
        """Get list of all metric column names"""
        metric_columns = [
            f'volume_{days}d_{metric}'
            for days in self.time_bins
            for metric in self.metrics
        ]
        ratio_columns = [
            f'volume_{metric}_{from_d}d_to_{to_d}d_ratio'
            for from_d, to_d in self.ratio_pairs
            for metric in ['avg', 'count'] if metric in self.metrics
        ]
        return metric_columns + ratio_columns

# Example usage
def example_usage():
    # Create sample data
    data = {
        'customer_id': ['A1', 'A1', 'A1', 'A2', 'A2'],
        'amount': [100, 200, 150, 300, 250],
        'account_age': [10, 20, 30, 15, 25]  # normalized days
    }
    df = pd.DataFrame(data)
    
    # Initialize calculator with custom configuration
    calculator = TransactionStatsCalculator(
        time_bins=[7, 14, 30],
        metrics=['avg', 'std', 'count'],
        ratio_pairs=[(7, 14), (7, 30), (14, 30)]
    )
    
    # Process transactions
    result_df = calculator.process_transactions(
        df=df,
        amount_col='amount',
        customer_id_col='customer_id',
        age_col='account_age'
    )
    
    # Get column names
    columns = calculator.get_column_names()
    
    return result_df, columns

example_usage()